- [Linear Regression](#linear-regression)
- [Logistic Regression + MCMC + Moment Matching](#Logistic-Regression-+-MCMC-+-Moment-Matching)
- [Logistic Regression + MCMC](#Logistic-Regression-+-MCMC)
- [Logistic Regression + Search](#Logistic-Regression-+-Search)

In [1]:
import probpy as pp
import numpy as np
import numba
import random

linear regression
---

In [2]:
%%time
def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]

w = [5, -2, 1, 5] # True underlying model
x = np.random.rand(100, 3) * 10
y = predict(w, x) + pp.normal.sample(mu=0, sigma=1, size=100).reshape(-1)



prior = pp.multivariate_normal.med(mu=np.ones(4) * 0, sigma=np.eye(4) * 5)
likelihood = pp.unilinear.med(sigma=1) # There exist an implementation for linear because it has a conjugate prior

for i in range(100):
    data = (y[i], x[i])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, prior=prior)
    
    if i % 10 == 0:
        w_approx = pp.mode(prior)
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - predict(w_approx, x)).mean(), 
              "True MSE", np.square(y - predict(w, x)).mean())
        print()

Parameter Estimate [0.62945907 1.04030399 1.38943747 0.22187135]
Prior MSE 236.14026486427062 True MSE 0.9295563918386928

Parameter Estimate [ 5.03384098 -2.04242046  1.0458533   4.49254238]
Prior MSE 1.1156529019110772 True MSE 0.9295563918386928

Parameter Estimate [ 5.00171183 -2.10335714  1.05217679  5.25890319]
Prior MSE 0.9547183417513909 True MSE 0.9295563918386928

Parameter Estimate [ 4.94934162 -2.08692361  1.03290793  5.46006742]
Prior MSE 0.8942972424078205 True MSE 0.9295563918386928

Parameter Estimate [ 4.88098409 -2.04564108  0.98936908  5.68943923]
Prior MSE 0.8946674022992412 True MSE 0.9295563918386928

Parameter Estimate [ 4.90174701 -2.06784802  0.98461191  5.80234427]
Prior MSE 0.8618938961334188 True MSE 0.9295563918386928

Parameter Estimate [ 4.91712289 -2.07114437  0.9580543   5.9014963 ]
Prior MSE 0.8525450886358604 True MSE 0.9295563918386928

Parameter Estimate [ 4.92216242 -2.06329766  0.95407032  5.88746845]
Prior MSE 0.848502279680264 True MSE 0.9295563

In [3]:
@numba.jit(nopython=True, fastmath=True, forceobj=False)
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

fast_p = pp.normal.fast_p # Need to assign here first since numba does not support jitting methods of classes

def likelihood(y, x, w):
    return fast_p(y - sigmoid(np.sum(x * w[:-1]) + w[-1]), mu=0.0, sigma=1.0)

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]

Logistic Regression + MCMC + Moment Matching
---

In [4]:
%%time



    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(6):
    j = random.randint(0, 80)
    data = (y[j: j + 20], x[j: j + 20])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   prior=prior, 
                                   match_moments_for=pp.multivariate_normal,
                                   batch=5,
                                   samples=50000,
                                   mixing=5000, 
                                   energy=0.1,
                                   mode="mcmc")

    w_approx = pp.mode(prior)
    print("Parameter Estimate", w_approx)

    print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
          "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
    print()

/usr/local/lib/python3.7/site-packages/probpy-0.0.1-py3.7.egg/probpy/learn/posterior/common.py:61: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, C), readonly array(float64, 2d, A))


Parameter Estimate [-0.97009623  0.27894465  1.05471155 -0.28060688]
Prior MSE 0.06349766407435388 True MSE 0.028442062980429297

Parameter Estimate [-0.89313312  0.49276941  1.98401066 -0.75166448]
Prior MSE 0.055511721564717925 True MSE 0.028442062980429297

Parameter Estimate [-1.36706553  0.81945303  1.96599013 -0.91297171]
Prior MSE 0.04191596831938002 True MSE 0.028442062980429297

Parameter Estimate [-1.50176096  0.91431854  1.97971899 -1.00719531]
Prior MSE 0.03954184849132962 True MSE 0.028442062980429297

Parameter Estimate [-1.48843031  1.09389861  2.28148444 -1.23420324]
Prior MSE 0.03641346380960732 True MSE 0.028442062980429297

Parameter Estimate [-1.40940583  1.10857199  2.59032608 -1.42629829]
Prior MSE 0.03560933710458337 True MSE 0.028442062980429297

CPU times: user 21.9 s, sys: 64.4 ms, total: 21.9 s
Wall time: 21.9 s


Logistic Regression + MCMC
---

In [9]:
%%time

w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(5):
    data = (y, x)
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   prior=prior, 
                                   batch=5,
                                   samples=20000,
                                   mixing=18000, 
                                   energy=0.3,
                                   mode="mcmc")

    modes = pp.mode(prior) # modes are sorted in order first is largest

    print("Number of modes", len(modes))
    w_approx = modes[0]

    print("Parameter Estimate", w_approx)

    print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
          "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
    print()

Number of modes 1
Parameter Estimate [-0.40251571  1.63657293  2.64594073 -1.89631666]
Prior MSE 0.043379478686802544 True MSE 0.029320377157292802

Number of modes 1
Parameter Estimate [-1.64726359  1.24265392  4.02096962 -2.03267843]
Prior MSE 0.03318426198006277 True MSE 0.029320377157292802

Number of modes 1
Parameter Estimate [-1.59024907  1.82567696  4.04794259 -2.38693076]
Prior MSE 0.030119804944745553 True MSE 0.029320377157292802

Number of modes 1
Parameter Estimate [-2.14826314  2.05129974  5.11420032 -2.44792898]
Prior MSE 0.03274770047208002 True MSE 0.029320377157292802

Number of modes 1
Parameter Estimate [-2.67868997  2.6145071   5.67260813 -2.71680115]
Prior MSE 0.03226325696568973 True MSE 0.029320377157292802

CPU times: user 39.8 s, sys: 11.9 ms, total: 39.8 s
Wall time: 39.8 s


Logistic Regression + Search
---


In [5]:
%%time

    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 10)

for i in range(5):
    data = (y, x)
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   prior=prior, 
                                   batch=50,
                                   samples=1000,
                                   energy=0.25,
                                   mode="search",
                                   volume=1000)

    modes = pp.mode(prior) # modes are sorted in order first is largest

    
    print("Number of modes", len(modes))
    for mode in modes:
        print(mode)

    w_approx = modes[0]

    print("Parameter Estimate", w_approx)

    print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
          "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
    print()

Number of modes 1
[-1.69907879  1.35527693  2.60255812 -1.31164715]
Parameter Estimate [-1.69907879  1.35527693  2.60255812 -1.31164715]
Prior MSE 0.03183061146749331 True MSE 0.02340749888029968



Exception: generic function did not have a numba probability